In [2]:
import secrets
import time
import random
import hashlib
import base58


import ecdsa
import binascii

import codecs

import struct

In [94]:
# wallet.py

class KeyGenerator():
    def __init__(self):
        self.CURVE_ORDER = int('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141', 16)
        self.POOL_SIZE = 256
        self.KEY_BYTES = 32
        self.pool = [0] * self.POOL_SIZE
        self.pool_pointer = 0
        self.prng_state = None
        self.__init_pool()
        
    def __init_pool(self):
        for i in range(self.POOL_SIZE):
            random_byte = secrets.randbits(8)
            self.__seed_byte(random_byte)
        time_int = int(time.time())
        self.__seed_int(time_int)
    
    def __seed_int(self, n):
        self.__seed_byte(n)
        self.__seed_byte(n >> 8)
        self.__seed_byte(n >> 16)
        self.__seed_byte(n >> 24)
    
    def __seed_byte(self, n):
        self.pool[self.pool_pointer] ^= n & 255
        self.pool_pointer += 1
        if self.pool_pointer >= self.POOL_SIZE:
            self.pool_pointer = 0
    
    def seed_input(self, str_input):
        time_int = int(time.time())
        self.__seed_int(time_int)
        for char in str_input:
            char_code = ord(char)
            self.__seed_byte(char_code)

    def generate_key(self):
        big_int = self.__generate_big_int()
        big_int %= (self.CURVE_ORDER - 1)
        big_int += 1
        key = hex(big_int)[2:]
        return key
    
    def __generate_big_int(self):
        if self.prng_state is None:
            seed = int.from_bytes(self.pool, byteorder='big', signed=False)
            random.seed(seed)
            self.prng_state = random.getstate()
            random.setstate(self.prng_state)
            big_int = random.getrandbits(self.KEY_BYTES * 8)
            self.prng_state = random.getstate()
            return big_int

class Wallet:
    def __init__(self, seed=None, key_to_file=False):
        self.__create_priv_key(seed, key_to_file)
        self.public_keys = []

    def __create_priv_key(self, seed=None, key_to_file=False):
        keygen = KeyGenerator()
        if seed:
            keygen.seed_input(seed)
        key = keygen.generate_key()
        if key_to_file:
            f = open("privkey.pem", "w+")
            f.write(key)
            f.close()
        else:
            print(key)
        del key
        del keygen

    @staticmethod
    def priv_to_WIF(filename):
        f = open(filename, "r")
        if f.mode == "r":
            key = f.read()
            print(key)
            _priv = key.upper()
            priv_add_x80 = "80" + _priv
            sh = sha256(sha256(priv_add_x80))
            first_4_bytes = sh[0:8]
            resulting_hex = priv_add_x80 + first_4_bytes
            result_wif = Wallet.base58(resulting_hex)
            del key
            return result_wif
        else: print ("Incorrect file!")
    
    @staticmethod
    def base58(address_hex):
        alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
        b58_string = ''
        leading_zeros = len(address_hex) - len(address_hex.lstrip('0'))
        address_int = int(address_hex, 16)
        while address_int > 0:
            digit = address_int % 58
            digit_char = alphabet[digit]
            b58_string = digit_char + b58_string
            address_int //= 58
        ones = leading_zeros // 2
        for one in range(ones):
            b58_string = '1' + b58_string
        return b58_string
    
    @staticmethod
    def private_to_public(private_key, is_print=False):
        pk_bytes = codecs.decode(private_key, 'hex')
        
        key = ecdsa.SigningKey.from_string(pk_bytes, curve=ecdsa.SECP256k1).verifying_key
        key_bytes = key.to_string()
        key_hex = codecs.encode(key_bytes, 'hex')
        btc_byte = b'04'
        public_key = btc_byte + key_hex
        
        if is_print:
            print(public_key.decode('utf-8'))
        
        return public_key.decode('utf-8')
    
    @staticmethod
    def public_key_to_addr(key):
        public_key_bytes = codecs.decode(key, 'hex')

        # Run SHA256 for the public key
        sha256_bpk = hashlib.sha256(public_key_bytes)
        sha256_bpk_digest = sha256_bpk.digest()

        # Run ripemd160 for the SHA256
        ripemd160_bpk = hashlib.new('ripemd160')
        ripemd160_bpk.update(sha256_bpk_digest)
        ripemd160_bpk_digest = ripemd160_bpk.digest()
        ripemd160_bpk_hex = codecs.encode(ripemd160_bpk_digest, 'hex')

        # Add network byte
        network_byte = b'00'
        network_bitcoin_public_key = network_byte + ripemd160_bpk_hex
        network_bitcoin_public_key_bytes = codecs.decode(network_bitcoin_public_key, 'hex')

        # Double SHA256 to get checksum
        sha256_2_nbpk_digest = hashlib.sha256(hashlib.sha256(network_bitcoin_public_key_bytes).digest()).digest()
        sha256_2_hex = codecs.encode(sha256_2_nbpk_digest, 'hex')
        checksum = sha256_2_hex[:8]

        # Concatenate public key and checksum to get the address
        address_hex = (network_bitcoin_public_key + checksum).decode('utf-8')
        address = Wallet.base58(address_hex)
        return address
    
    @staticmethod
    def private_key_to_addr(key):
        return Wallet.public_key_to_addr(private_to_public(key))
    
    @staticmethod
    def sign_message(message, private_key):
        key_bytes = codecs.decode(private_key, 'hex')
        sk = ecdsa.SigningKey.from_string(key_bytes, curve=ecdsa.SECP256k1)
        signed_msg = sk.sign(message.encode('utf-8'))
        return (signed_msg.hex(), Wallet.private_to_public(private_key))
    
    @staticmethod
    def verify_message(message, public_key, signature):
        vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key[2:]), curve=ecdsa.SECP256k1)
        return (vk.verify(bytes.fromhex(signature), message.encode('utf-8')))

In [100]:
class Transaction:

    def __init__(self, sender_address=None, recepient_address=None, amount=None):
        if not sender_address:
            raise ValueError("No sender address !")
        if not recepient_address:
            raise ValueError("No recepient address !")
        if not amount:
            raise ValueError("No amount !")
        if amount <= 0:
            raise ValueError("Amount must be > 0 !")

        self.sender_address = sender_address
        self.recepient_address = recepient_address
        self.amount = amount

    def to_dict(self):
        return dict({'sender_address': self.sender_address,
                            'recipient_address': self.recipient_address,
                            'value': self.value})

    def transaction_hash(self):
        tsx = self.sender_address + self.recepient_address + '%04x' % self.amount
        tsx_bytes = codecs.encode(bytes(tsx, encoding='utf-8'), 'hex')
        transaction_hash = hashlib.sha256(tsx_bytes).digest()
        return transaction_hash.hex()

class CoinbaseTransaction(Transaction):
    def __init__(self, amount=50):
        super(CoinbaseTransaction, self).__init__(
            '0000000000000000000000000000000000',
            '0000000000000000000000000000000000',
            amount)


In [96]:
class Serializer:
    @staticmethod
    def serialize(num_of_coins, sender_addr, recepient_addr, sender_public_key, signature):
        return (
            '%04x' % num_of_coins +
            sender_addr +
            recepient_addr +
            sender_public_key +
            signature
        )

In [97]:
class Deserializer:
    @staticmethod
    def deserialize(transaction):
        num_of_coins = int(transaction[0:4], 16)
        send_addr = transaction[4:38]
        recep_addr = transaction[38:72]
        pub_key = transaction[72:202]
        signt = transaction[202:]
        return dict({
            'num_of_coins' : num_of_coins,
            'sender_addr' : send_addr,
            'recepient_addr' : recep_addr,
            'public_key' : pub_key,
            'signature' : signt
                    }) 

In [7]:
def address_check(addr):
    def decode_base58(bc, length):
        digits58 = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
        n = 0
        for char in bc:
            n = n * 58 + digits58.index(char)
        return n.to_bytes(length, 'big')
    bcbytes = decode_base58(addr, 25)
    return bcbytes[-4:] == hashlib.sha256(hashlib.sha256(bcbytes[:-4]).digest()).digest()[:4]


def transaction_validation(transaction, transaction_hash):
    
    if  not address_check(transaction['sender_addr'])                       or \
        not address_check(transaction['recepient_addr'])                    or \
        Wallet.public_key_to_addr(transaction['public_key']) != sender_addr or \
        not Wallet.verify_message(transaction_hash,
                                  transaction['public_key'],
                                  transaction['signature']):
            return False
    return True

In [8]:
sender_wallet = Wallet(key_to_file=False)

1793b5d912e4d0c29104daed77af21cc962d81182926077cd92aa3d213131e1d


In [9]:
recepient_wallet = Wallet(key_to_file=False)

478ad104e1f41deb556b1cb0cf6138b8d98daa34dc9a83fc07c3f87b33ba8ffb


In [10]:
sender_prk = '75ba85e367262e9c5e8198e7917d1992bcaeaa3f1d046e413088d7cf4d134737'
sender_pbk = sender_wallet.private_to_public(sender_prk)
sender_addr = sender_wallet.public_key_to_addr(sender_pbk)

recepient_prk = '29c9ef9ab3713f59a7d54f1101272085641459310bcfc25e61f5edc394f1736a'
recepient_pbk = recepient_wallet.private_to_public(recepient_prk)
recepient_addr = sender_wallet.public_key_to_addr(recepient_pbk)

In [11]:
print(sender_pbk)
print(len(sender_pbk))

04dbaa85101b6e47c0caa603a84f308a5d86eff6722ba4db524890b1b0a5f3d1d2d9e2f61accc4daa2c7310517c7fce4dd4b638b5de7f2bbe650061da9a042276f
130


In [12]:
def testDeserialize():
    serialized = '000f1K3tSuwpkjXpnvE6XNnS1kDcPWGTbTMEG11KahXwTdYJL2vefijiuLGezBvCrdFtt9vb04dbaa85101b6e47c0caa603a84f308a5d86eff6722ba4db524890b1b0a5f3d1d2d9e2f61accc4daa2c7310517c7fce4dd4b638b5de7f2bbe650061da9a042276f1aef90bec11a983c528fa25417dc078aa3b067bb5a3e0c19660e0931bdb34876f02a1e58e9a943e2796199a49ffb8fbfbc08bb696f19571ce3e6796c21b991a7'
    deserialized = Deserializer.deserialize(serialized)
    
    assert deserialized['num_of_coins'] == 15
    assert deserialized['sender_addr'] == '1K3tSuwpkjXpnvE6XNnS1kDcPWGTbTMEG1'
    assert deserialized['recepient_addr'] == '1KahXwTdYJL2vefijiuLGezBvCrdFtt9vb'
    assert deserialized['public_key'] == '04dbaa85101b6e47c0caa603a84f308a5d86eff6722ba4db524890b1b0a5f3d1d2d9e2f61accc4daa2c7310517c7fce4dd4b638b5de7f2bbe650061da9a042276f'
    assert deserialized['signature'] == '1aef90bec11a983c528fa25417dc078aa3b067bb5a3e0c19660e0931bdb34876f02a1e58e9a943e2796199a49ffb8fbfbc08bb696f19571ce3e6796c21b991a7'

In [13]:
# pending_pool.py
class PendingPool:
    def __init__(self):
        self.transaction_pool = []

    def add_transaction_to_pool(self, serialized_transaction):
        deserialized = Deserializer.deserialize(serialized_transaction)
        transaction = Transaction(deserialized['sender_addr'],
                                   deserialized['recepient_addr'],
                                   deserialized['num_of_coins'])
        transaction_hash = transaction.transaction_hash()
        transaction_validation(deserialized, transaction_hash)
        self.transaction_pool.append(serialized_transaction)
    
    def return_last_transactions(self, num_of_transactions=3):
            return self.transaction_pool[-num_of_transactions:]

In [57]:
# merkle.py

def merkle(hashList):
    if len(hashList) == 1:
        return hashList[0]
    newHashList = []
    for i in range(0, len(hashList)-1, 2):
        newHashList.append(hash2(hashList[i], hashList[i+1]))
    if len(hashList) % 2 == 1:
        newHashList.append(hash2(hashList[-1], hashList[-1]))
    return merkle(newHashList)

def hash2(a, b):
    s = (a + b).encode('utf-8')
    h = hashlib.sha256(hashlib.sha256(s).digest()).digest()
    return h.hex()

In [20]:
serialized_tsx = '000f1K3tSuwpkjXpnvE6XNnS1kDcPWGTbTMEG11KahXwTdYJL2vefijiuLGezBvCrdFtt9vb04dbaa85101b6e47c0caa603a84f308a5d86eff6722ba4db524890b1b0a5f3d1d2d9e2f61accc4daa2c7310517c7fce4dd4b638b5de7f2bbe650061da9a042276f1aef90bec11a983c528fa25417dc078aa3b067bb5a3e0c19660e0931bdb34876f02a1e58e9a943e2796199a49ffb8fbfbc08bb696f19571ce3e6796c21b991a7'
serialized_tsx1 = '00011K3tSuwpkjXpnvE6XNnS1kDcPWGTbTMEG11KahXwTdYJL2vefijiuLGezBvCrdFtt9vb04dbaa85101b6e47c0caa603a84f308a5d86eff6722ba4db524890b1b0a5f3d1d2d9e2f61accc4daa2c7310517c7fce4dd4b638b5de7f2bbe650061da9a042276f1aef90bec11a983c528fa25417dc078aa3b067bb5a3e0c19660e0931bdb34876f02a1e58e9a943e2796199a49ffb8fbfbc08bb696f19571ce3e6796c21b991a7'

signature, publ_k = Wallet.sign_message(transaction_hash, sender_prk)

In [21]:
serialized_tsx = Serializer.serialize(15, sender_addr, recepient_addr, sender_pbk, signature)

In [22]:
deserialized = Deserializer.deserialize(serialized_tsx)

In [19]:
transaction_hash = Transaction(deserialized['sender_addr'],
                                   deserialized['recepient_addr'],
                                   deserialized['num_of_coins']).transaction_hash()

In [23]:
pool = PendingPool()

In [24]:
pool.add_transaction_to_pool(serialized_tsx)
pool.add_transaction_to_pool(serialized_tsx)
pool.add_transaction_to_pool(serialized_tsx)

In [25]:
pool_tsxs = pool.return_last_transactions()

In [26]:
print(len(pool_tsxs))

3


In [69]:
merkle_hash = merkle(pool_tsxs)

In [70]:
print(merkle_hash)

df03c4435cd982f8bb28267f8648e59c9c699a8f09003463b7d40220e4840b7a


In [102]:
# block.py

class Block:
    def __init__(self, timestamp, previous_hash, transactions, nonce=0):
        self.timestamp = timestamp
        self.nonce = nonce
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.merkle_root = merkle(transactions)
        self.block_hash = self.hash_block()
    
    def set_nonse(self, nonce):
        self.nonce = nonce

    def hash_block(self):
        s = (
            str(self.timestamp) +
            str(self.nonce) +
            str(self.previous_hash) +
            "".join(self.transactions) +
            self.merkle_root
        ).encode('utf-8')
        hash_s = hashlib.sha256(s).digest().hex()
        return hash_s
    
    def to_dict(self):
        return {
            'timestamp' : self.timestamp,
            'nonce' : self.nonce,
            'previous_hash' : self.previous_hash,
            'transactions' : self.transactions,
            'merkle_root' : self.merkle_root,
            'block_hash' : self.block_hash
        }
    def validate_transactions(self, transactions):
        if transactions:
            for transaction in transactions:
                deserialized = Deserializer.deserialize(transaction)
                tsx = Transaction(deserialized['sender_addr'],
                                   deserialized['recepient_addr'],
                                   deserialized['num_of_coins'])
                tsx_hash = tsx.transaction_hash()
                transaction_validation(deserialized, tsx_hash)

In [90]:
block = Block(12592875, merkle_hash, pool_tsxs)

In [88]:
print(str(309535820) + 'aaaa')

309535820aaaa


In [98]:
# blockchain.py

class Blockchain:
    def __init__(self):
        self.complexity = 2
        self.chain = [self.genesis_block()]

    def check_hash(self, h):
        if int(h[0:self.complexity]) == 0:
            return True
        return False
    def mine(self, block):
        nonce = 0
        h = block.hash_block()
        while not check_hash(h):
            nonce += 1
            block.set_nonse(nonce)
            h = block.hash_block()
        if check_hash(h):
            self.chain.append(block)
    
    def genesis_block(self):
        prev_hash = '0'
        tsx = CoinbaseTransaction()
        
        serialize(num_of_coins, sender_addr, recepient_addr, sender_public_key, signature)
        
        self.sender_address = sender_address
        self.recepient_address = recepient_address
        serialized_tsx = Serialize(tsx.amount, tsx.sender_address, tsx.recepient_address, )
        genesis = Block(0, prev_hash, transaction)
        return genesis
    
    def is_valid_chain(self):
        for block in blocks:
            block.validate_transactions()

In [103]:
blockchain = Blockchain()

TypeError: object of type 'CoinbaseTransaction' has no len()